# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vernon,50.2581,-119.2691,33.60,87,100,1.01,CA,1704853742
1,1,tazovsky,67.4667,78.7000,-21.68,98,13,9.08,RU,1704853743
2,2,lompoc,34.6391,-120.4579,56.79,87,0,14.97,US,1704853561
3,3,college,64.8569,-147.8028,4.06,82,100,0.00,US,1704853747
4,4,codrington,-38.2667,141.9667,68.70,57,10,11.61,AU,1704853747


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(x='Lng', y='Lat', geo=True, size='humidity', alpha=0.7, color='blue', hover_cols=['humidity'])

# Display the map


:Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
cities_filtered = city_data_df.loc[(city_data_df['Max Temp'] <= 27) & (city_data_df['Max Temp'] >= 22)]
cities_filtered = cities_filtered.loc[(city_data_df['Wind Speed'] >= 4.5)]
cities_filtered = cities_filtered.loc[(city_data_df['Cloudiness'] >= 0)]


# Drop any rows with null values
cities_filtered.dropna()

# Display sample data
cities_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,olonkinbyen,70.9221,-8.7187,26.51,94,100,14.97,SJ,1704853865
118,118,mussoorie,30.4500,78.0833,25.77,27,0,5.84,IN,1704853957
173,173,vadso,70.0744,29.7487,23.16,93,100,8.05,NO,1704854048
202,202,lingen,52.5167,7.3167,22.98,39,0,10.80,DE,1704854093
218,218,nemuro,43.3236,145.5750,26.46,68,60,19.08,JP,1704854113
261,261,aktau,43.6500,51.2000,23.58,63,88,20.13,KZ,1704854171
294,294,schweich,49.8222,6.7526,25.52,61,0,5.03,DE,1704854214
309,309,yulinshi,38.2906,109.7494,25.92,64,0,6.73,CN,1704854239
350,350,princeville,45.7668,-61.2985,24.42,79,100,6.91,CA,1704854298
421,421,sault ste. marie,46.5168,-84.3333,25.36,86,100,13.80,CA,1704854163


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df, columns=["City", "Country", "Lat","Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vernon,CA,50.2581,-119.2691,87,
1,tazovsky,RU,67.4667,78.7000,98,
2,lompoc,US,34.6391,-120.4579,87,
3,college,US,64.8569,-147.8028,82,
4,codrington,AU,-38.2667,141.9667,57,
...,...,...,...,...,...,...
540,keflavik,IS,64.0049,-22.5624,76,
541,krasnovishersk,RU,60.4078,57.0820,100,
542,lubango,AO,-14.9172,13.4925,94,
543,kayangel,PW,8.0823,134.7173,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius" : radius,
    "apiKey" : geoapify_key,
    "filter" : '',
    "bias" : ''
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = ["Lat"]
    lng = ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'latitude={lat},longitude={lng}'

    params["bias"] = f'latitude={lat},longitude={lng}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vernon - nearest hotel: No hotel found
tazovsky - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
college - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
kolonia town - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
ulladulla - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
gokceada - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
amga - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
fort saskatchewan - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
padang -

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vernon,CA,50.2581,-119.2691,87,No hotel found
1,tazovsky,RU,67.4667,78.7000,98,No hotel found
2,lompoc,US,34.6391,-120.4579,87,No hotel found
3,college,US,64.8569,-147.8028,82,No hotel found
4,codrington,AU,-38.2667,141.9667,57,No hotel found
...,...,...,...,...,...,...
540,keflavik,IS,64.0049,-22.5624,76,No hotel found
541,krasnovishersk,RU,60.4078,57.0820,100,No hotel found
542,lubango,AO,-14.9172,13.4925,94,No hotel found
543,kayangel,PW,8.0823,134.7173,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(x='Lng', y='Lat', geo=True, size='humidity', alpha=0.7, color='blue', hover_cols=['Hotel Name', 'Country'])


# Display the map
